In [ ]:
from chebGreen.greenlearning.utils import DataProcessor
from chebGreen.greenlearning.model import *
from chebGreen.backend import plt
from chebGreen.utils import runCustomScript

# %env TF_MLC_LOGGING=1
# %env TF_CPP_MIN_LOG_LEVEL=2

In [ ]:
%%time
example = 'laplace'
script = 'generate_example'
theta = None
Nsample = 500
lmbda = 0.01
Nf = 500
Nu = 500
noise_level = 0
runCustomScript(script,example,theta,Nsample,lmbda,Nf,Nu,noise_level)

In [ ]:
data = DataProcessor(f"datasets/{example}/data.mat")
data.generateDataset(trainRatio = 0.95)

In [ ]:
F = data.trainDataset[0].cpu().numpy()
U = data.trainDataset[1].cpu().numpy()

In [ ]:
A = F.T

In [ ]:
u, s, v = np.linalg.svd(A)

In [ ]:
plt.figure()
plt.plot(A[:,0])

In [ ]:
np.linalg.matrix_rank(A)

In [ ]:
plt.figure()
plt.plot(u[:,3])

In [ ]:
plt.figure()
plt.plot(F[0,:])

In [ ]:
plt.figure()
plt.plot(U[0,:])

In [ ]:
model = GreenNN()
dimension = 1
domain = [0,1,0,1]
layerConfig = [50,50,50,50]
activation = 'rational'
dirichletBC = True
model.build(dimension, domain, layerConfig, activation, dirichletBC)

In [ ]:
%%time
lossHistory = model.train(data, epochs = {'adam':int(5000), 'lbfgs':int(0)})

In [ ]:
xF, xU = data.xF, data.xU
x, y = np.meshgrid(xU, xF)
G = model.evaluateG(x,y)
N = model.evaluateN(data.xF)

In [ ]:
plt.figure()
plt.semilogy(range(len(lossHistory['training'])), lossHistory['training'], 'b', alpha = 0.5)
plt.semilogy(range(len(lossHistory['validation'])), lossHistory['validation'], 'r--', alpha = 0.5)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Training','Validation'])

In [ ]:
fig = plt.figure(figsize = (8,6))
plt.contourf(x,y,G, 50, cmap = 'turbo', vmin = np.min(G), vmax = np.max(G))
plt.colorbar()

In [ ]:
model.saveModels(f'savedModels/{example}')

In [ ]:
savedModel = GreenNN()
savedModel.build(dimension, domain, layerConfig, activation, dirichletBC, loadPath = f'savedModels/{example}')

In [ ]:
from chebGreen.chebpy2 import Chebfun2
from chebGreen.chebpy2.chebpy.core.algorithms import chebpts2
from chebGreen.chebpy2.chebpy.api import chebfun

In [ ]:
%%time
cheb2prefs = Chebpy2Preferences()
cheb2prefs.prefx.eps = 1e-6
cheb2prefs.prefx.eps = 1e-6
g = Chebfun2(savedModel.evaluateG, domain = domain, prefs = cheb2prefs, simplify = False)

In [ ]:
def vec2cheb(f, x):
    f = f.reshape((-1))
    x = x.reshape((-1))
    domain = [np.min(x), np.max(x)] # Compute the bounds of the domain
    
    # Function is resampled at twice as many sampled points to maintain accuracy in a different basis.
    N = 2 * f.shape[0] # Check if this is fine
    
    # Compute the Chebyshev nodes and scale them to the domain
    xc = (chebpts2(N) + 1) * ((domain[1] - domain[0])/2) + domain[0]

    # Compute the interpolated value of the function at the Chebyshev nodes
    fc = np.interp(xc, x, f).reshape((-1,1))

    return chebfun(fc, domain)

In [ ]:
def computeEmpiricalError(data, G, N = None):
    RE, UC, U0 = [],[],[]
    if N is None:
        print('Assuming a zero homogeneous solution.')
    for i in range(data.valDataset[1].cpu().numpy().shape[0]):
        xF, xU = data.xF, data.xU
        f, u  = data.valDataset[0].cpu().numpy()[i,:], data.valDataset[1].cpu().numpy()[i,:]
        f0, u0 = vec2cheb(f,xF), vec2cheb(u,xU)
        if N is None:
            uc = G.T.integralTransform(f0)
        else:
            uc = G.T.integralTransform(f0) + N
        re = (uc - u0).abs()/u0.abs().sum()
        RE.append(re)
    
    error = np.mean([re.sum() for re in RE])
    return error

In [ ]:
error = computeEmpiricalError(data, g, N = None)

In [ ]:
%%time
example = 'laplace'
script = 'generate_example'
theta = None
Nsample = 100
lmbda = 0.01
Nf = 500
Nu = 500
noise_level = 0
runCustomScript(script,example,theta,Nsample,lmbda,Nf,Nu,noise_level)

In [ ]:
data = DataProcessor(f"datasets/{example}/data.mat")
data.generateDataset(trainRatio = 0.1)

In [ ]:
%%time
error = computeEmpiricalError(data, g, N = None)

In [ ]:
error

In [ ]:
xx = np.linspace(domain[2],domain[3],2000)
yy = np.linspace(domain[0],domain[1],2000)
x, y = np.meshgrid(xx,yy)
G = savedModel.evaluateG(x,y)
N = savedModel.evaluateN(xx)

In [ ]:
def green(x,s):
    g = 0
    g = (x <= s) * (x * (1-s)) + (x > s) * (s * (1-x))
    # np.random.seed(42)
    # g = g + np.random.normal(loc = 0.0, scale = 0.25 * eps * factor, size=x.shape)
    return g

In [ ]:
fig = plt.figure(figsize = (8,7))

vmin, vmax = np.min(G), np.max(G)

levels = np.linspace(vmin, vmax, 50, endpoint = True)
plt.contourf(x,y,G, levels = levels, cmap = 'turbo', vmin = vmin, vmax = vmax)

ticks = np.linspace(vmin, vmax, 10, endpoint=True)
cbar = plt.colorbar(ticks = ticks, fraction = 0.046, pad = 0.04)
cbar.ax.set_yticklabels(["{:4.2f}".format(i) for i in ticks])

plt.xlabel(f'$x$', fontsize = 14)
plt.ylabel(f'$s$', rotation='horizontal', labelpad=11, fontsize = 14)
# plt.title("Data", fontsize = 14)
# fig.savefig(f'G.png', dpi = fig.dpi)

In [ ]:
Gexact = green(x,y)

fig = plt.figure(figsize = (8,7))

vmin, vmax = np.min(Gexact), np.max(Gexact)

levels = np.linspace(vmin, vmax, 50, endpoint = True)
plt.contourf(x,y,Gexact, levels = levels, cmap = 'turbo', vmin = vmin, vmax = vmax)

ticks = np.linspace(vmin, vmax, 10, endpoint=True)
cbar = plt.colorbar(ticks = ticks, fraction = 0.046, pad = 0.04)
cbar.ax.set_yticklabels(["{:4.2f}".format(i) for i in ticks])

plt.xlabel(f'$x$', fontsize = 14)
plt.ylabel(f'$s$', rotation='horizontal', labelpad=11, fontsize = 14)
# plt.title("Data", fontsize = 14)

In [ ]:
fig = plt.figure(figsize = (8,7))
plt.plot(xx,N)
plt.xlabel(f'$x$', fontsize = 14)
plt.ylabel(f'$u_{{hom}}$', rotation='horizontal', labelpad=11, fontsize = 14)
plt.xlim([0,1])
fig.savefig(f'u_hom.png', dpi = fig.dpi)

In [ ]:
np.sqrt(np.linalg.norm(G-Gexact)/np.linalg.norm(Gexact))

In [ ]:
fig = plt.figure(figsize = (8,6))
plt.contourf(x,y,np.abs(G-Gexact), 50, cmap = 'turbo')
plt.colorbar()